In [3]:
"""
Simple example script fitting a model for synthetic data
"""
from __future__ import print_function
import jax.numpy as jnp
from jax import jit, random
import time
import matplotlib.pyplot as plt
import seaborn as sns

from codebase.vi.BBVI import DiagMvnBBVI
from codebase.gsmooth.opt import FilteredOptimization
from codebase.gsmooth.smoothers import AdamFilter
import CV as cvg
import models

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#########################################
# construct model function + vb object  #
#########################################
# lnpdf, D = make_model()
lnpdf, D, name = models.set_lnpdf("frisk")
key = random.PRNGKey(0)
th0 = random.normal(key, (D,))
print(lnpdf(th0))  # example use

# create bbvi object --- this just keeps references to lnpdf,
# grad(lnpdf), hvp(lnpdf), etc
vbobj = DiagMvnBBVI(lnpdf, D, lnpdf_is_vectorized=False)

# initialize params
key = random.PRNGKey(1)
lam0 = random.normal(key, (vbobj.num_variational_params,)) * 0.01 - 1
lam0 = lam0.at[D:].set(-3.)

TypeError: true_fun output and false_fun output must have identical types, got
DIFFERENT ShapedArray(float32[]) vs. ShapedArray(float32[1]).